### Recommendation System

In [9]:
import math

# Content-based: Books dataset
books = [
    {"title": "Pride and Prejudice", "genre": "Romance", "author": "Austen"},
    {"title": "Sense and Sensibility", "genre": "Romance", "author": "Austen"},
    {"title": "The Da Vinci Code", "genre": "Thriller", "author": "Brown"},
    {"title": "Angels and Demons", "genre": "Thriller", "author": "Brown"},
    {"title": "The Shining", "genre": "Horror", "author": "King"}
]

# Collaborative filtering: Movies ratings dataset
user_ratings = {
    "Alice": {"Movie1": 5, "Movie2": 3, "Movie3": 4},
    "Bob": {"Movie1": 4, "Movie2": 5, "Movie3": 2},
    "Carol": {"Movie2": 4, "Movie3": 5, "Movie4": 3},
    "Dave": {"Movie1": 2, "Movie3": 5, "Movie4": 4}
}

#### Content-Based Recommendation(Books)

In [10]:
# Feature vector: genre + author
def book_vector(book):
    return [book["genre"], book["author"]]

def cosine_similarity_features(f1, f2):
    all_features = list(set(f1 + f2))
    vec1 = [1 if feat in f1 else 0 for feat in all_features]
    vec2 = [1 if feat in f2 else 0 for feat in all_features]
    
    dot = sum(a*b for a,b in zip(vec1, vec2))
    mag1 = math.sqrt(sum(a*a for a in vec1))
    mag2 = math.sqrt(sum(b*b for b in vec2))
    if mag1==0 or mag2==0:
        return 0
    return dot / (mag1*mag2)

def recommend_books(target_title, top_n=2):
    target_book = next(book for book in books if book["title"] == target_title)
    sims = []
    for book in books:
        if book["title"] == target_title:
            continue
        sim = cosine_similarity_features(book_vector(target_book), book_vector(book))
        sims.append((sim, book["title"]))
    sims.sort(reverse=True)
    return [title for _, title in sims[:top_n]]

print("Content-based recommendations for 'Pride and Prejudice':", recommend_books("Pride and Prejudice"))
print("Content-based recommendations for 'The Da Vinci Code':", recommend_books("The Da Vinci Code"))


Content-based recommendations for 'Pride and Prejudice': ['Sense and Sensibility', 'The Shining']
Content-based recommendations for 'The Da Vinci Code': ['Angels and Demons', 'The Shining']


#### Collaborative Filtering(Movies)

In [12]:
# Cosine similarity between users
def cosine_similarity_users(u1, u2):
    common_movies = set(u1.keys()) & set(u2.keys())
    if not common_movies:
        return 0
    vec1 = [u1[m] for m in common_movies]
    vec2 = [u2[m] for m in common_movies]
    dot = sum(a*b for a,b in zip(vec1, vec2))
    mag1 = math.sqrt(sum(a*a for a in vec1))
    mag2 = math.sqrt(sum(b*b for b in vec2))
    if mag1==0 or mag2==0:
        return 0
    return dot / (mag1*mag2)

# Predict rating for a user and movie
def predict_rating(user, movie, k=2):
    sims = []
    for other_user, ratings in user_ratings.items():
        if other_user == user or movie not in ratings:
            continue
        sim = cosine_similarity_users(user_ratings[user], ratings)
        sims.append((sim, ratings[movie]))
    sims.sort(reverse=True)
    top_k = sims[:k]
    if not top_k:
        return None
    numerator = sum(sim*rating for sim,rating in top_k)
    denominator = sum(sim for sim,_ in top_k)
    return numerator/denominator if denominator != 0 else None

def recommend_movies(user, top_n=2):
    all_movies = set(m for r in user_ratings.values() for m in r)
    unrated = [m for m in all_movies if m not in user_ratings[user]]
    predictions = [(predict_rating(user, m), m) for m in unrated]
    predictions = [(p,m) for p,m in predictions if p is not None]
    predictions.sort(reverse=True)
    return [m for _, m in predictions[:top_n]]

print("Collaborative filtering recommendations for Alice:", recommend_movies("Alice"))
print("Collaborative filtering recommendations for Carol:", recommend_movies("Carol"))


Collaborative filtering recommendations for Alice: ['Movie4']
Collaborative filtering recommendations for Carol: ['Movie1']
